<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/246_PredRevenue_Gap_Orchestrator_Tier2_BizLogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate Revenue Impact

In [ ]:
def calculate_revenue_impact_score(
    gap: Dict[str, Any],
    max_gap_amount: float = 1000.0
) -> float:
    """
    Calculate revenue impact score (0-10 scale).

    Args:
        gap: Gap dictionary
        max_gap_amount: Maximum gap amount for normalization

    Returns:
        Score from 0-10
    """
    gap_amount = abs(gap.get("gap_amount", 0.0))

    # Normalize to 0-10 scale
    if max_gap_amount > 0:
        score = min(10.0, (gap_amount / max_gap_amount) * 10.0)
    else:
        score = 0.0

    return round(score, 2)



# ✅ **Function: `calculate_revenue_impact_score`**

**Purpose:** convert the raw *gap amount* into a normalized **0–10 score** representing how financially important the gap is.

---

## 🔍 **What the function does (briefly)**

1. **Reads the gap amount**

   ```python
   gap_amount = abs(gap.get("gap_amount", 0.0))
   ```

2. **Normalizes it to a 0–10 scale** based on the largest gap in the dataset

   ```python
   score = min(10.0, (gap_amount / max_gap_amount) * 10.0)
   ```

3. **Caps the score at 10**

4. **Returns a rounded float** (e.g., 7.3)

---

# 🧠 Why this matters (business + agent design perspective)

### **1. Turns dollars into an actionable, comparable score**

\$500 loss vs. \$50 loss
→ not directly comparable when ranking gaps
→ but 8.2 vs. 1.4 is.

This is key to prioritization.

---

### **2. Normalizes across customers and segments**

If one customer normally spends \$30/week and another spends \$500/week:

* A \$30 gap is *catastrophic* for customer A
* Barely noticeable for customer B

Normalization handles this.

---

### **3. Fits perfectly into multi-objective scoring**

Revenue Impact is just **one dimension** of an orchestrator’s decision:

* Revenue Impact
* Churn Risk
* Customer Value
* Recovery Probability

Together they form a **decision engine**, which is exactly what you're building.

---

### **4. Keeps scoring transparent and tunable**

A business can change:

* what the max expected gap amount is
* how aggressively to score
* which metrics matter more

All without changing core code.

---

# 🧪 Example

If the largest gap is **\$1,000** and this customer’s gap is **\$400**:

```
400 / 1000 = 0.4 → score = 4.0
```

If the largest gap is **\$200**, but this gap is **\$200**:

```
200 / 200 = 1.0 → score = 10.0
```

This is why normalization matters for fairness.

---

# 🎯 Key takeaways

### ✔ Simple

But extremely powerful.

### ✔ Converts raw financials → prioritization score

This is the backbone of a decision engine.

### ✔ Works for any industry

Retail, SaaS, B2B, healthcare, logistics.

### ✔ Transparent

No ML black box. Executives love it.

### ✔ Extensible

You can always swap normalization for ML-based weighting later.



# Calculate Churn Risk

In [ ]:
def calculate_churn_risk_score(
    gap: Dict[str, Any],
    churn_risk_data: Optional[Dict[str, Any]] = None
) -> float:
    """
    Calculate churn risk score (0-10 scale).

    Args:
        gap: Gap dictionary
        churn_risk_data: Churn risk data for the customer (if available)

    Returns:
        Score from 0-10
    """
    # Check gap type for churn indicators
    gap_type = gap.get("gap_type", "")
    weeks_at_risk = gap.get("weeks_at_risk", 0)

    # Zero spend gaps are high churn risk
    if gap_type == "zero_spend":
        # More weeks = higher risk
        score = min(10.0, 7.0 + (weeks_at_risk * 1.0))
    elif churn_risk_data:
        # Use churn risk score if available
        churn_score = churn_risk_data.get("churn_risk_score", 0.0)
        score = churn_score * 10.0
    elif gap_type in ["declining_revenue", "below_baseline"]:
        # Declining revenue indicates some churn risk
        gap_percentage = abs(gap.get("gap_percentage", 0.0))
        score = min(10.0, (gap_percentage / 50.0) * 10.0)  # 50% gap = max risk
    else:
        score = 0.0

    return round(score, 2)


# ✅ **`calculate_churn_risk_score` — What It Does (Concise Summary)**

This function **assigns a churn-risk score (0–10)** to a revenue gap.
It decides *how strongly this gap implies churn risk* based on:

1. **Gap type** (zero spend? declining revenue?)
2. **Weeks at risk** (how long the customer has been inactive)
3. **Optional churn model output** (if provided)

It always returns a number between **0 (no churn risk)** and **10 (maximum churn risk)**.

---

# 🔍 **Breakdown of the Logic**

## **1. Zero-Spend Gaps → HIGH churn risk**

If a customer has a gap type `zero_spend`, that’s the most dangerous signal.

```python
if gap_type == "zero_spend":
    score = min(10.0, 7.0 + (weeks_at_risk * 1.0))
```

Meaning:

* Base score = **7**
* +1 point for every additional week of zero spend
* Capped at 10
* So 3 zero weeks → 9
* 4+ zero weeks → 10

**Business meaning:**
If they’ve stopped buying, churn becomes the default assumption.

---

## **2. If we have churn model data → use it**

If the orchestrator also ran `detect_churn_risk`, then it uses that:

```python
churn_score = churn_risk_data.get("churn_risk_score", 0.0)
score = churn_score * 10.0
```

E.g.,
Churn model score = 0.67 → final score = **6.7**

**Business meaning:**
Use the most accurate signal available.

---

## **3. Declining or below baseline gaps → some churn risk**

If there’s NO churn model data and NO zero-spend, but revenue is falling:

```python
gap_percentage = abs(gap.get("gap_percentage", 0.0))
score = min(10.0, (gap_percentage / 50.0) * 10.0)
```

* A 25% drop → score = 5
* A 50% drop → score = 10
* A 10% drop → score = 2

**Business meaning:**
If customers spend less, churn risk increases proportionally.

---

## **4. Otherwise → low/no churn risk**

Fallback:

```python
score = 0.0
```

---

# 🧠 **Why This Matters for the Orchestrator**

You should learn that:

### ✔ This is a **rule-based risk engine**

No ML required, but the logic is still powerful.

### ✔ It blends signals from different sources

* Gap type
* Severity
* Optional churn-risk model output

This is exactly what **decision engines** do in real companies.

### ✔ It maps raw signals → a unified 0–10 scale

This normalization is key for:

* Ranking
* Prioritization
* Action selection

### ✔ Easily extensible

You could add:

* “Price sensitivity score”
* “Satisfaction score”
* “Support tickets score”
* “Store experience score”

…without breaking the system.

---

You now see how modern orchestrators:

* Take raw data
* Transform them into **signals**
* Convert signals into **scores**
* Use scores to **prioritize action**

This is **decision engineering**, and Data Scientists who master this will lead the next decade.



# Calculate Customer Value Score

In [ ]:
def calculate_customer_value_score(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    all_customers_baselines: Dict[str, Dict[str, Any]]
) -> float:
    """
    Calculate customer value score (0-10 scale).

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline for this customer
        all_customers_baselines: All customers' baselines (for comparison)

    Returns:
        Score from 0-10
    """
    if not all_customers_baselines:
        return 5.0  # Default middle score

    # Get customer's total revenue
    customer_total = revenue_baseline.get("total_revenue", 0.0)

    # Calculate percentile
    all_totals = [
        baseline.get("total_revenue", 0.0)
        for baseline in all_customers_baselines.values()
    ]

    if not all_totals:
        return 5.0

    # Find percentile
    sorted_totals = sorted(all_totals)
    customer_rank = sum(1 for total in sorted_totals if total <= customer_total)
    percentile = (customer_rank / len(sorted_totals)) * 100

    # Convert percentile to 0-10 score
    # Top 10% = 10, bottom 10% = 0
    if percentile >= 90:
        score = 10.0
    elif percentile >= 75:
        score = 8.0
    elif percentile >= 50:
        score = 6.0
    elif percentile >= 25:
        score = 4.0
    else:
        score = 2.0

    return round(score, 2)


# ✅ **What This Function Does (In Plain English)**

This function scores **how valuable a customer is** relative to all other customers, on a **0–10 scale**.

It uses **percentiles** based on *total revenue*.

High revenue → high percentile → high score
Low revenue → low percentile → low score

This is a **relative value ranking**, not an absolute one.

---

# 🔍 **Key Components (What You Should Learn)**

### **1. Customer value is *relative*, not absolute.**

This is important:

* A customer spending \$500 may be “high value” in a low-spend business
* The same \$500 may be “low value” in a luxury service

Using percentile ranking internally adapts to the customer base.

---

### **2. Percentile → Score Mapping is Rule-Based**

The rules:

| Percentile | Score | Interpretation      |
| ---------- | ----- | ------------------- |
| ≥ 90       | 10    | VIP / Highest value |
| ≥ 75       | 8     | Highly valuable     |
| ≥ 50       | 6     | Above average       |
| ≥ 25       | 4     | Below average       |
| < 25       | 2     | Low value           |

This is business logic — transparent, editable, interpretable.

A business could change those breakpoints with **no ML retraining**.

---

### **3. Easy to Extend**

You could add more sophistication later:

* RFM scoring (Recency, Frequency, Monetary)
* Customer lifetime value prediction
* Cohort-based value comparison
* Margin-based contribution instead of revenue

But for an orchestrator MVP?

Percentile ranking is **perfect**:

* stable
* cheap
* interpretable
* robust to outliers

---

# 🧠 **From a Data Scientist Perspective (What to Learn)**

### ✔ **Percentile-based scoring is a powerful DS tool**

This function teaches you a high-value lesson:

> **Relative scoring is often more actionable than absolute metrics.**

This applies to:

* anomaly detection
* prioritization
* customer segmentation
* risk scoring
* ranking tasks

Percentiles eliminate noise and scale differences.

---

### ✔ **This is Decision Engineering**

This function is a classic example of:

**“Convert raw metrics → into a decision score.”**

You’re turning:

* revenue
  → percentile
  → priority score
  → (later) orchestrator actions

This is exactly what modern AI agents need.

---

# 🏗 **From an Orchestrator Architect Perspective**

This function is designed to be:

### **Modular**

You can swap it out easily.

### **Composable**

It plugs into the final scoring engine without touching other code.

### **Explainable**

You can include in a report:

> “Customer is in the 92nd percentile for revenue → Value score: 10.”

No hallucination. No black box. Clean reasoning.

### **Deterministic**

Same input → same output every time
Perfect for enterprise use cases.


